In [ ]:
import pandas as pd
import os, inspect
import numpy as np
import pandas as pd

In [ ]:
#현재 접근 폴더 경로 가져오기
NOW =os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) #현재 파일이 위치한 경로

if NOW == '/content': # 구글 드라이브일 경우 접근 허용 및 경로 탐색
  from google.colab import drive
  drive.mount('/content/drive')
  finalPath = os.path.join(NOW,'drive/MyDrive/Colab Notebooks/Final') # 
  NOW = finalPath # 임시

Mounted at /content/drive


### 데이터 불러오기

In [ ]:
DATAPATH  = os.path.join(NOW,'DATA') #DATA 폴더 접근
TRAINPATH = os.path.join(DATAPATH,'00_source/train.csv') #train데이터 경로 찾기
TESTPATH  = os.path.join(DATAPATH,'00_source/test.csv') # test데이터 경로 찾기
train = pd.read_csv(TRAINPATH, sep=',') #train 데이터 가져오기
test = pd.read_csv(TESTPATH,sep=',') # test 데이터 가져오기

### test 데이터 정렬

In [ ]:
# 샘플 데이터 가져오기
SAMPLEPATH = os.path.join(DATAPATH,'sample_submission.csv') # 경로 찾기
Sample = pd.read_csv(SAMPLEPATH,sep = ',') #sample 데이터 가져오기

#정렬
sorted_test=pd.DataFrame(columns=test.columns)
for i in range(len(Sample)):
  sorted_test.loc[i]=list(test[test['id']==Sample.iloc[i,0]].iloc[0,:]) # sample 데이터의 id를 기준으로 test 데이터를 정렬하기 -> 더 좋은 코드 있나 연구!!!!
test=sorted_test

KeyboardInterrupt: ignored

### 데이터 결측치 및 중복 데이터 제거

In [ ]:
Data=train
#데이터 결측치 및 중복 데이터 제거하기
#결측된 데이터 제거
Data.dropna()
#중복
Data.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

print(Data.shape)

(33600, 514)


### k-Fold split하기

In [ ]:
#type별로 데이터 나누기
Noise=Data[Data['leaktype']=='noise']
In=Data[Data['leaktype']=='in']
Out=Data[Data['leaktype']=='out']
Normal=Data[Data['leaktype']=='normal']
Other=Data[Data['leaktype']=='other']

# 각 데이터에 대하여 셔플하기
Random = 777
Noise = Noise.sample(frac=1, random_state = Random).reset_index(drop=True)
In = In.sample(frac=1, random_state = Random).reset_index(drop=True)
Out = Out.sample(frac=1, random_state = Random).reset_index(drop=True)
Normal = Normal.sample(frac=1, random_state = Random).reset_index(drop=True)
Other = Other.sample(frac=1, random_state = Random).reset_index(drop=True)

#데이터 K-Fold로 split하기
Fold = 10

#Noise
TempData=Noise
FoldDataNo = int(len(TempData)/Fold)
for i in range(Fold):
  SilceData = TempData.iloc[FoldDataNo*i : FoldDataNo*(i+1),:]
  s1="Noise_Fold_%d = SilceData" %(i+1)
  exec(s1)

#In
TempData=In
FoldDataNo = int(len(TempData)/Fold)
for i in range(Fold):
  SilceData = TempData.iloc[FoldDataNo*i : FoldDataNo*(i+1),:]
  s1="In_Fold_%d = SilceData" %(i+1)
  exec(s1)

#Out
TempData=Out
FoldDataNo = int(len(TempData)/Fold)
for i in range(Fold):
  SilceData = TempData.iloc[FoldDataNo*i : FoldDataNo*(i+1),:]
  s1="Out_Fold_%d = SilceData" %(i+1)
  exec(s1)  

#Normal
TempData=Normal
FoldDataNo = int(len(TempData)/Fold)
for i in range(Fold):
  SilceData = TempData.iloc[FoldDataNo*i : FoldDataNo*(i+1),:]
  s1="Normal_Fold_%d = SilceData" %(i+1)
  exec(s1)  

#Other
TempData=Other
FoldDataNo = int(len(TempData)/Fold)
for i in range(Fold):
  SilceData = TempData.iloc[FoldDataNo*i : FoldDataNo*(i+1),:]
  s1="Other_Fold_%d = SilceData" %(i+1)
  exec(s1)  

#데이터 클래스 별로 나눈 것들을 하나로 합치기
for i in range(Fold):
  s1="Combine_%d = pd.concat([Noise_Fold_%d , In_Fold_%d , Out_Fold_%d , Normal_Fold_%d , Other_Fold_%d ])" %(i+1,i+1,i+1,i+1,i+1,i+1)
  exec(s1)

###  ANN Labeling

Noise  = [1,0,0,0,0], 
In     = [0,1,0,0,0],
Out    = [0,0,1,0,0],
Normal = [0,0,0,1,0],
Other  = [0,0,0,0,1]

In [ ]:
Temp_Label = pd.DataFrame()
for i in range(Fold):
  s1= "TempData = Combine_%d['leaktype']" %(i+1) #leaktype 얻기
  exec(s1)
  s1= "TrainLabel_%d = pd.DataFrame()" %(i+1) # Label 데이터 생성
  exec(s1)

  #Labeling
  for j in range(len(TempData)):
    if TempData.iloc[j]=='noise':
      Temp_Label=pd.DataFrame([1,0,0,0,0]).transpose() # Noise  = [1,0,0,0,0]
    elif TempData.iloc[j]=='in':
      Temp_Label=pd.DataFrame([0,1,0,0,0]).transpose() # In = [0,1,0,0,0]
    elif TempData.iloc[j]=='out':
      Temp_Label=pd.DataFrame([0,0,1,0,0]).transpose() # Out = [0,0,1,0,0]
    elif TempData.iloc[j]=='normal':
      Temp_Label=pd.DataFrame([0,0,0,1,0]).transpose() # Normal = [0,0,0,1,0]
    elif TempData.iloc[j]=='other':
      Temp_Label=pd.DataFrame([0,0,0,0,1]).transpose() # Other = [0,0,0,0,1]

    #합치기
    s2="TrainLabel_%d =pd.concat([TrainLabel_%d ,Temp_Label],ignore_index=True)" %(i+1,i+1)
    exec(s2)

  #leaktype 삭제
  s1="Combine_%d = Combine_%d.drop(['leaktype'],axis='columns')" %(i+1,i+1)
  exec(s1)

### k-fold 된 데이터, ANN Labeling과 정렬된 test 데이터 저장하기

In [ ]:
#TrainData
SAVETRAINPATH = os.path.join(DATAPATH,'01_split')
for i in range(Fold):
  s1="Combine_%d.to_csv(SAVETRAINPATH +'/k_Fold_%d.csv',index=False)" %(i+1,i+1)
  exec(s1)
  s2="TrainLabel_%d.to_csv(SAVETRAINPATH +'/k_FoldTrainLabel_%d.csv',index=False)" %(i+1,i+1)
  exec(s2)

#testdata
test.to_csv(SAVETRAINPATH +'/test.csv',index=False)